In [ ]:
!pip install --quiet datasets
!pip install --quiet frozendict
!pip install --quiet transformers
!pip install --quiet qatch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 38.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.3/38.3 MB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 337.4/337.4 kB 30.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.4/345.4 kB 30.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==1.5.3, but you have pandas 2.2.1 which is incompatib

In [ ]:
import warnings
import os
import pandas as pd
from google.colab import drive
import pickle
import torch
import numpy as np
from transformers import TapasTokenizer


import collections
import numpy as np
from collections import defaultdict

from transformers import TapasConfig, TapasForQuestionAnswering
from torch.optim import AdamW

from torch.optim.lr_scheduler import ReduceLROnPlateau
from qatch import MetricEvaluator
from qatch.database_reader import MultipleDatabases
import logging

# Set logging level to ERROR to suppress warnings
logging.basicConfig(level=logging.ERROR)

warnings.filterwarnings("ignore", category=FutureWarning)


In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# The path to multiple databases
db_save_path = '/content/drive/MyDrive/spider/test_database'
databases = MultipleDatabases(db_save_path)
evaluator = MetricEvaluator(databases=databases)

In [ ]:
with open('/content/drive/MyDrive/spider/db_to_tables.pkl', 'rb') as file:
    db_to_tables = pickle.load(file)

with open('/content/drive/MyDrive/spider/db_to_tables_constraint.pkl', 'rb') as file:
    db_to_tables_constraint = pickle.load(file)

with open('/content/drive/MyDrive/spider/db_table_to_df.pkl', 'rb') as file:
    db_table_to_df = pickle.load(file)

In [ ]:
with open('/content/drive/MyDrive/spider/db_to_tables_valid.pkl', 'rb') as file:
    db_to_tables_valid = pickle.load(file)

with open('/content/drive/MyDrive/spider/db_to_tables_constraint_valid.pkl', 'rb') as file:
    db_to_tables_constraint_valid = pickle.load(file)

with open('/content/drive/MyDrive/spider/db_table_to_df_valid.pkl', 'rb') as file:
    db_table_to_df_valid = pickle.load(file)

In [ ]:
# initialize the tokenizer
tokenizer = TapasTokenizer.from_pretrained("google/tapas-base")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/490 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/262k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/154 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.52k [00:00<?, ?B/s]

In [ ]:
class TableDataset(torch.utils.data.Dataset):
    def __init__(self, df, tokenizer):
        self.df = df
        self.tokenizer = tokenizer
        self.problematic_ids = []  # Initialize a list to store IDs of problematic questions

    def __getitem__(self, idx):
        item = self.df.iloc[idx]
        table_name = item['seq_id']
        try:
            table = db_table_to_df[table_name].astype(str)
            encoding = self.tokenizer(
                table=table,
                queries=item.question,
                answer_coordinates=item.answer_coordinates,
                answer_text=item.answer_text,
                truncation=True,
                padding="max_length",
                return_tensors="pt",
            )
            float_answer = torch.tensor(item.float_value)
            encoding = {key: val.squeeze(0) for key, val in encoding.items()}
            encoding["float_answer"] = float_answer


        except Exception as e:
            print(f"Error processing index {idx} (ID: {item['ID']}): {e}")
            self.problematic_ids.append(item.ID)  # Store the ID of the problematic question
            encoding = None

        return encoding


    def __len__(self):
        return len(self.df)

    def get_problematic_ids(self):
        # Returns the list of problematic IDs
        return self.problematic_ids

# Custom collate function to filter out None values, if not already implemented
def custom_collate_fn(batch):
    batch = [item for item in batch if item is not None]  # Filter out None values
    if not batch:
        return None
    return torch.utils.data.dataloader.default_collate(batch)

In [ ]:
spider_train_pd = pd.read_pickle('/content/drive/MyDrive/spider/SPIDER_Simple_cleaned_final.pkl')
spider_train_pd=spider_train_pd.drop(columns=["query_toks","query_toks_no_value","question_toks","operator"])

spider_valid_pd = pd.read_pickle('/content/drive/MyDrive/spider/SPIDER_Simple_cleaned_final_valid.pkl')


qatch_pickle = pd.read_pickle('/content/drive/MyDrive/spider/QATCH_SPIDER.pkl')

percentage = 0.1
sampled_df = qatch_pickle.groupby('sql_tags', group_keys=False).apply(lambda x: x.sample(frac=percentage, random_state=42))

sampled_df=sampled_df.drop(columns=["sql_tags"])

train_df = pd.concat([spider_train_pd, sampled_df], axis=0)
# train_df=spider_train_pd.copy()

train_df=train_df.drop(columns=["ID"])
train_df.reset_index(inplace=True)
train_df.rename(columns={'index': 'ID'}, inplace=True)

<ipython-input-9-b6ccab1eba53>:10: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  sampled_df = qatch_pickle.groupby('sql_tags', group_keys=False).apply(lambda x: x.sample(frac=percentage, random_state=42))


In [ ]:
train_df

,ID,answer_text,db_id,query,question,table_used,seq_id,answer_coordinates,float_value
0,0,[5],department_management,SELECT count(*) FROM head WHERE age > 56,How many heads of the departments are older th...,head,department_management_X_head,"[(4, 0)]",5.0
1,1,"[Pádraig Harrington, Connecticut, 43.0, Stewar...",department_management,"SELECT name , born_state , age FROM head ORD...","List the name, born state and age of the heads...",head,department_management_X_head,"[(8, 1), (8, 2), (8, 3), (6, 1), (6, 2), (6, 3...",NaN
2,2,"[1789, State, 9.96, 1789, Treasury, 11.1, 1947...",department_management,"SELECT creation , name , budget_in_billions ...","List the creation year, name and budget of eac...",department,department_management_X_department,"[(0, 2), (0, 1), (0, 4), (1, 2), (1, 1), (1, 4...",NaN
3,4,"[Tiger Woods, K. J. Choi, Jeff Maggert, Stewar...",department_management,SELECT name FROM head WHERE born_state != 'Cal...,What are the names of the heads who are born o...,head,department_management_X_head,"[(0, 1), (2, 1), (4, 1), (6, 1), (8, 1), (9, 1)]",NaN
4,5,[California],department_management,SELECT born_state FROM head GROUP BY born_stat...,What are the names of the states where at leas...,head,department_management_X_head,"[(1, 2)]",NaN
...,...,...,...,...,...,...,...,...,...
3582,10314,"[2, 6, 1, advanced database, advanced database...",e_learning,"SELECT * FROM ""Courses"" WHERE ""course_id"" > 1","Show the data of the table ""Courses"" where ""co...",Courses,e_learning_X_Courses,"[(1, 0), (1, 1), (0, 0), (1, 3), (1, 4), (2, 0...",NaN
3583,13268,"[1729, 1729, England Premier League, 4769, 476...",soccer_1,"SELECT * FROM ""League"" WHERE ""id"" > 1","Show the data of the table ""League"" where ""id""...",League,soccer_1_X_League,"[(1, 0), (1, 1), (1, 2), (2, 0), (2, 1), (2, 2...",NaN
3584,21105,"[1, 7, 3, 1978-04-15 04:49:18, None, None, 2, ...",behavior_monitoring,"SELECT * FROM ""Assessment_Notes"" WHERE ""teache...","Show the data of the table ""Assessment_Notes"" ...",Assessment_Notes,behavior_monitoring_X_Assessment_Notes,"[(0, 0), (0, 1), (0, 2), (0, 3), (0, 4), (0, 5...",NaN
3585,6501,"[0.0, 2, leader, 1981-10-04 22:44:50, 1985-05-...",tracking_grants_for_research,"SELECT * FROM ""Project_Staff"" WHERE ""project_i...","Show the data of the table ""Project_Staff"" whe...",Project_Staff,tracking_grants_for_research_X_Project_Staff,"[(0, 0), (0, 1), (0, 2), (0, 3), (0, 4), (0, 5...",NaN


In [ ]:
train_dataset = TableDataset(df=train_df, tokenizer=tokenizer)
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=8, collate_fn=custom_collate_fn)

for idx, batch in enumerate(train_dataloader):
    if batch is not None:
        continue
    else:
        print(f"Batch {idx} skipped due to problematic data")

print(train_dataset.get_problematic_ids())


prob=train_dataset.get_problematic_ids()
train_df = train_df[~train_df['ID'].isin(prob)]
train_dataset = TableDataset(df=train_df, tokenizer=tokenizer)
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=8, collate_fn=custom_collate_fn)

Error processing index 769 (ID: 884): Couldn't find all answers
Error processing index 770 (ID: 885): Couldn't find all answers
Error processing index 771 (ID: 886): Couldn't find all answers
Error processing index 772 (ID: 887): Couldn't find all answers
Error processing index 781 (ID: 896): Couldn't find all answers
Error processing index 782 (ID: 897): Couldn't find all answers
Error processing index 839 (ID: 968): Couldn't find all answers
Error processing index 840 (ID: 969): Couldn't find all answers
Error processing index 847 (ID: 980): Couldn't find all answers
Error processing index 848 (ID: 981): Couldn't find all answers
Error processing index 1125 (ID: 1326): Couldn't find all answers
Error processing index 1126 (ID: 1327): Couldn't find all answers
Error processing index 1137 (ID: 1338): Couldn't find all answers
Error processing index 1138 (ID: 1339): Couldn't find all answers
Error processing index 1196 (ID: 1411): Couldn't find all answers
Error processing index 2251 (I

In [ ]:
# with open('/content/drive/MyDrive/spider/train_dataloader_spider_2000qatch.pkl', 'wb') as f:
#     pickle.dump(train_dataloader, f)
with open('/content/drive/MyDrive/spider/train_dataloader_spider_2000qatch.pkl', 'rb') as file:
    train_dataloader = pickle.load(file)

In [ ]:
spider_valid_pd = spider_valid_pd.reset_index().rename(columns={'table_used':'tbl_name'})

In [ ]:
#@title Computing predictions
def compute_prediction_sequence(model, data, device):
  """Computes predictions using model's answers to the previous questions."""

  # prepare data
  input_ids = data["input_ids"].to(device)
  attention_mask = data["attention_mask"].to(device)
  token_type_ids = data["token_type_ids"].to(device)

  all_logits = []
  prev_answers = None

  num_batch = data["input_ids"].shape[0]

  for idx in range(num_batch):

    if prev_answers is not None:
        coords_to_answer = prev_answers[idx]
        # Next, set the label ids predicted by the model
        prev_label_ids_example = token_type_ids_example[:,3] # shape (seq_len,)
        model_label_ids = np.zeros_like(prev_label_ids_example.cpu().numpy()) # shape (seq_len,)

        # for each token in the sequence:
        token_type_ids_example = token_type_ids[idx] # shape (seq_len, 7)
        for i in range(model_label_ids.shape[0]):
          segment_id = token_type_ids_example[:,0].tolist()[i]
          col_id = token_type_ids_example[:,1].tolist()[i] - 1
          row_id = token_type_ids_example[:,2].tolist()[i] - 1
          if row_id >= 0 and col_id >= 0 and segment_id == 1:
            model_label_ids[i] = int(coords_to_answer[(col_id, row_id)])

        # set the prev label ids of the example (shape (1, seq_len) )
        token_type_ids_example[:,3] = torch.from_numpy(model_label_ids).type(torch.long).to(device)

    prev_answers = {}
    # get the example
    input_ids_example = input_ids[idx] # shape (seq_len,)
    attention_mask_example = attention_mask[idx] # shape (seq_len,)
    token_type_ids_example = token_type_ids[idx] # shape (seq_len, 7)
    # forward pass to obtain the logits
    outputs = model(input_ids=input_ids_example.unsqueeze(0),
                    attention_mask=attention_mask_example.unsqueeze(0),
                    token_type_ids=token_type_ids_example.unsqueeze(0))
    logits = outputs.logits
    all_logits.append(logits)

    # convert logits to probabilities (which are of shape (1, seq_len))
    dist_per_token = torch.distributions.Bernoulli(logits=logits)
    probabilities = dist_per_token.probs * attention_mask_example.type(torch.float32).to(dist_per_token.probs.device)

    # Compute average probability per cell, aggregating over tokens.
    # Dictionary maps coordinates to a list of one or more probabilities
    coords_to_probs = collections.defaultdict(list)
    prev_answers = {}
    for i, p in enumerate(probabilities.squeeze().tolist()):
      segment_id = token_type_ids_example[:,0].tolist()[i]
      col = token_type_ids_example[:,1].tolist()[i] - 1
      row = token_type_ids_example[:,2].tolist()[i] - 1
      if col >= 0 and row >= 0 and segment_id == 1:
        coords_to_probs[(col, row)].append(p)

    # Next, map cell coordinates to 1 or 0 (depending on whether the mean prob of all cell tokens is > 0.5)
    coords_to_answer = {}
    for key in coords_to_probs:
      coords_to_answer[key] = np.array(coords_to_probs[key]).mean() > 0.5
    prev_answers[idx+1] = coords_to_answer

  logits_batch = torch.cat(tuple(all_logits), 0)

  return logits_batch


def getAnswer(row):
    row_index = row['ID']
    print(f"Processing row {row_index}...")

    seq_id = row["db_id"] + "_X_" + row["tbl_name"]
    if seq_id not in db_table_to_df or db_table_to_df[seq_id].empty:
        print(f"Row {row_index} skipped: table not found or empty.")
        return None

    table = db_table_to_df[seq_id]
    table = table.astype(str)
    query_nl = [row["question"]]

    try:
        inputs = tokenizer(table=table, queries=query_nl, padding='max_length', return_tensors="pt", truncation=True)
        logits = compute_prediction_sequence(model, inputs, 'cuda')
        predicted_answer_coordinates, = tokenizer.convert_logits_to_predictions(inputs, logits.cpu().detach())
        answers = []
        for tbl_cords in predicted_answer_coordinates:
            query_answer = defaultdict(list)
            # For each coordinate set in the predicted query coordinates,
            # construct a dictionary of row-wise query answers.
            [query_answer[row].append(table.iat[(row, col)])
             for row, col in tbl_cords]
            answers.extend(list(query_answer.values()))
        print(f"Row {row_index} processed successfully.")
    except IndexError:
        print(f"Row {row_index} skipped due to IndexError.")
        return None

    return answers

def getAnswerValid(row):
    row_index = row['ID']
    # print(f"Processing row {row_index}...")

    seq_id = row["db_id"] + "_X_" + row["tbl_name"]
    if seq_id not in db_table_to_df_valid or db_table_to_df_valid[seq_id].empty:
        print(f"Row {row_index} skipped: table not found or empty.")
        return None

    table = db_table_to_df_valid[seq_id]
    table = table.astype(str)
    query_nl = [row["question"]]

    try:
        inputs = row['input']
        logits = compute_prediction_sequence(model, inputs, 'cuda')
        predicted_answer_coordinates, = tokenizer.convert_logits_to_predictions(inputs, logits.cpu().detach())
        answers = []
        for tbl_cords in predicted_answer_coordinates:
            query_answer = defaultdict(list)
            # For each coordinate set in the predicted query coordinates,
            # construct a dictionary of row-wise query answers.
            [query_answer[row].append(table.iat[(row, col)])
             for row, col in tbl_cords]
            answers.extend(list(query_answer.values()))
        # print(f"Row {row_index} processed successfully.")
    except IndexError:
        # print(f"Row {row_index} skipped due to IndexError.")
        return None

    return answers

def getInput(row):
    row_index = row['ID']
    print(f"Processing row {row_index}...")

    seq_id = row["db_id"] + "_X_" + row["tbl_name"]
    if seq_id not in db_table_to_df_valid or db_table_to_df_valid[seq_id].empty:
        print(f"Row {row_index} skipped: table not found or empty.")
        return None

    table = db_table_to_df_valid[seq_id]
    table = table.astype(str)
    query_nl = [row["question"]]

    try:
        inputs = tokenizer(table=table, queries=query_nl, padding='max_length', return_tensors="pt", truncation=True)
        print(f"Row {row_index} processed successfully.")
    except IndexError:
        print(f"Row {row_index} skipped due to IndexError.")
        return None

    return inputs


In [ ]:
spider_valid_pd['input'] = spider_valid_pd.apply(getInput, axis=1)

Processing row 0...
Row 0 processed successfully.
Processing row 1...
Row 1 processed successfully.
Processing row 2...
Row 2 processed successfully.
Processing row 3...
Row 3 processed successfully.
Processing row 8...
Row 8 processed successfully.
Processing row 9...
Row 9 processed successfully.
Processing row 20...
Row 20 processed successfully.
Processing row 21...
Row 21 processed successfully.
Processing row 87...
Row 87 processed successfully.
Processing row 88...
Row 88 processed successfully.
Processing row 91...
Row 91 processed successfully.
Processing row 92...
Row 92 processed successfully.
Processing row 179...
Row 179 processed successfully.
Processing row 180...
Row 180 processed successfully.
Processing row 181...
Row 181 processed successfully.
Processing row 182...
Row 182 processed successfully.
Processing row 183...
Row 183 processed successfully.
Processing row 184...
Row 184 processed successfully.
Processing row 187...
Row 187 processed successfully.
Processing

In [ ]:
spider_valid_pd

,index,ID,answer_text,db_id,query,question,query_toks,query_toks_no_value,question_toks,tbl_name,operator,seq_id,input
0,0,0,[6],concert_singer,SELECT count(*) FROM singer,How many singers do we have?,"[SELECT, count, (, *, ), FROM, singer]","[select, count, (, *, ), from, singer]","[How, many, singers, do, we, have, ?]",singer,count,concert_singer_X_singer,"[input_ids, token_type_ids, attention_mask]"
1,1,1,[6],concert_singer,SELECT count(*) FROM singer,What is the total number of singers?,"[SELECT, count, (, *, ), FROM, singer]","[select, count, (, *, ), from, singer]","[What, is, the, total, number, of, singers, ?]",singer,count,concert_singer_X_singer,"[input_ids, token_type_ids, attention_mask]"
2,2,2,"[Joe Sharp, Netherlands, 52, John Nizinik, Fra...",concert_singer,"SELECT name , country , age FROM singer ORDE...","Show name, country, age for all singers ordere...","[SELECT, name, ,, country, ,, age, FROM, singe...","[select, name, ,, country, ,, age, from, singe...","[Show, name, ,, country, ,, age, for, all, sin...",singer,NaN,concert_singer_X_singer,"[input_ids, token_type_ids, attention_mask]"
3,3,3,"[Joe Sharp, Netherlands, 52, John Nizinik, Fra...",concert_singer,"SELECT name , country , age FROM singer ORDE...","What are the names, countries, and ages for ev...","[SELECT, name, ,, country, ,, age, FROM, singe...","[select, name, ,, country, ,, age, from, singe...","[What, are, the, names, ,, countries, ,, and, ...",singer,NaN,concert_singer_X_singer,"[input_ids, token_type_ids, attention_mask]"
4,4,8,"[Netherlands, United States, France]",concert_singer,SELECT DISTINCT country FROM singer WHERE age ...,What are all distinct countries where singers ...,"[SELECT, DISTINCT, country, FROM, singer, WHER...","[select, distinct, country, from, singer, wher...","[What, are, all, distinct, countries, where, s...",singer,NaN,concert_singer_X_singer,"[input_ids, token_type_ids, attention_mask]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
206,211,1005,"[1944.0, France, 1948.0, United States, 1949.0...",singer,"SELECT Birth_Year , Citizenship FROM singer",What are the birth years and citizenships of t...,"[SELECT, Birth_Year, ,, Citizenship, FROM, sin...","[select, birth_year, ,, citizenship, from, sin...","[What, are, the, birth, years, and, citizenshi...",singer,NaN,singer_X_singer,"[input_ids, token_type_ids, attention_mask]"
207,212,1006,"[Christy Walton, Alice Walton, Iris Fontbona, ...",singer,"SELECT Name FROM singer WHERE Citizenship != ""...",List the name of singers whose citizenship is ...,"[SELECT, Name, FROM, singer, WHERE, Citizenshi...","[select, name, from, singer, where, citizenshi...","[List, the, name, of, singers, whose, citizens...",singer,NaN,singer_X_singer,"[input_ids, token_type_ids, attention_mask]"
208,213,1007,"[Christy Walton, Alice Walton, Iris Fontbona, ...",singer,"SELECT Name FROM singer WHERE Citizenship != ""...",What are the names of the singers who are not ...,"[SELECT, Name, FROM, singer, WHERE, Citizenshi...","[select, name, from, singer, where, citizenshi...","[What, are, the, names, of, the, singers, who,...",singer,NaN,singer_X_singer,"[input_ids, token_type_ids, attention_mask]"
209,214,1008,"[Christy Walton, Alice Walton]",singer,SELECT Name FROM singer WHERE Birth_Year = 1...,Show the name of singers whose birth year is e...,"[SELECT, Name, FROM, singer, WHERE, Birth_Year...","[select, name, from, singer, where, birth_year...","[Show, the, name, of, singers, whose, birth, y...",singer,NaN,singer_X_singer,"[input_ids, token_type_ids, attention_mask]"


In [ ]:
# this is the default WTQ configuration
config = TapasConfig(
    num_aggregation_labels=4,# MAX MIN COUNT AVG
    use_answer_as_supervision=True,
    answer_loss_cutoff=0.664694,
    cell_selection_preference=0.207951,
    huber_loss_delta=0.121194,
    init_cell_selection_weights_to_zero=True,
    select_one_column=True,
    allow_empty_column_selection=False,
    temperature=0.0352513,
)
model = TapasForQuestionAnswering.from_pretrained("google/tapas-base", config=config)
optimizer = AdamW(model.parameters(), lr=5e-5)
model.to('cuda')

pytorch_model.bin:   0%|          | 0.00/443M [00:00<?, ?B/s]

Some weights of TapasForQuestionAnswering were not initialized from the model checkpoint at google/tapas-base and are newly initialized: ['aggregation_classifier.bias', 'aggregation_classifier.weight', 'column_output_bias', 'column_output_weights', 'output_bias', 'output_weights']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


TapasForQuestionAnswering(
  (tapas): TapasModel(
    (embeddings): TapasEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(1024, 768)
      (token_type_embeddings_0): Embedding(3, 768)
      (token_type_embeddings_1): Embedding(256, 768)
      (token_type_embeddings_2): Embedding(256, 768)
      (token_type_embeddings_3): Embedding(2, 768)
      (token_type_embeddings_4): Embedding(256, 768)
      (token_type_embeddings_5): Embedding(256, 768)
      (token_type_embeddings_6): Embedding(10, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): TapasEncoder(
      (layer): ModuleList(
        (0-11): 12 x TapasLayer(
          (attention): TapasAttention(
            (self): TapasSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias

In [ ]:
stats_columns = ['epoch', 'avg_loss', 'avg_cell_precision', 'avg_cell_recall', 'avg_tuple_cardinality', 'avg_tuple_constraint', 'avg_tuple_order']
stats_df = pd.DataFrame(columns=stats_columns)
# Create the scheduler
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=2, verbose=True)

# model.train()
for epoch in range(5):  # loop over the dataset multiple times
    print("Epoch:", epoch)
    epoch_loss = 0.0
    for batch in train_dataloader:
        # get the inputs;
        input_ids = batch["input_ids"].to('cuda')
        attention_mask = batch["attention_mask"].to('cuda')
        token_type_ids = batch["token_type_ids"].to('cuda')
        labels = batch["labels"].to('cuda')
        numeric_values = batch["numeric_values"].to('cuda')
        numeric_values_scale = batch["numeric_values_scale"].to('cuda')
        float_answer = batch["float_answer"].to('cuda')

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            labels=labels,
            numeric_values=numeric_values,
            numeric_values_scale=numeric_values_scale,
            float_answer=float_answer,
        )
        loss = outputs.loss
        print("Loss:", loss.item())
        epoch_loss += loss.item()

        loss.backward()
        optimizer.step()

    # Calculate average loss for the epoch
    avg_loss = epoch_loss / len(train_dataloader)

    # Update the learning rate based on the average loss
    scheduler.step(avg_loss)
    spider_valid_pd['predictions_TAPAS'] = spider_valid_pd.apply(getAnswerValid, axis=1)
    tests_df_results = evaluator.evaluate_with_df(spider_valid_pd,
                                      prediction_col_name=f'predictions_TAPAS',
                                      task="QA")

    mean_cell_precision = tests_df_results['cell_precision_predictions_TAPAS'].mean()
    mean_cell_recall = tests_df_results['cell_recall_predictions_TAPAS'].mean()
    mean_tuple_cardinality = tests_df_results['tuple_cardinality_predictions_TAPAS'].mean()
    mean_tuple_constraint = tests_df_results['tuple_constraint_predictions_TAPAS'].mean()
    mean_tuple_order = tests_df_results['tuple_order_predictions_TAPAS'].mean()

    print(f"Model Performance Metrics for epoch:{epoch}")
    print("----------------------------")
    print("Avg loss",avg_loss)
    print("Average Cell Precision:", mean_cell_precision)
    print("Average Cell Recall:", mean_cell_recall)
    print("Average Tuple Cardinality Accuracy:", mean_tuple_cardinality)
    print("Average Tuple Constraint Accuracy:", mean_tuple_constraint)
    print("Average Tuple Order Accuracy:", mean_tuple_order)
    print("----------------------------")
    # Append statistics for this epoch to the DataFrame
    epoch_stats_df = pd.DataFrame([{
        'epoch': epoch,
        'avg_loss': avg_loss,
        'avg_cell_precision': mean_cell_precision,
        'avg_cell_recall': mean_cell_recall,
        'avg_tuple_cardinality': mean_tuple_cardinality,
        'avg_tuple_constraint': mean_tuple_constraint,
        'avg_tuple_order': mean_tuple_order,
    }])
    stats_df = pd.concat([stats_df, epoch_stats_df], ignore_index=True)
stats_df.to_csv('/content/drive/MyDrive/model_checkpoints/stats_model_spiderQatch2000', index=False)

/usr/local/lib/python3.10/dist-packages/torch/optim/lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "


Epoch: 0
Loss: 0.4827987849712372
Loss: 0.37681007385253906
Loss: 0.2780929505825043
Loss: 0.015282493084669113
Loss: 0.20480428636074066
Loss: 0.1590030938386917
Loss: 0.1029558777809143
Loss: 1.3068126440048218
Loss: 0.3034680187702179
Loss: 0.3014427721500397
Loss: 0.7713432312011719
Loss: 0.15446807444095612
Loss: 0.3984731435775757
Loss: 0.14416226744651794
Loss: 0.0641130656003952
Loss: 0.4175359904766083
Loss: 0.17051322758197784
Loss: 0.36167797446250916
Loss: 0.5324742197990417
Loss: 0.2484447956085205
Loss: 0.16328707337379456
Loss: 0.4947841763496399
Loss: 0.2912781238555908
Loss: 0.2091706097126007
Loss: 0.27878156304359436
Loss: 0.04884657263755798
Loss: 0.20480778813362122
Loss: 0.1571599841117859
Loss: 0.18694321811199188
Loss: 0.0775856226682663
Loss: 0.19523897767066956
Loss: 0.2796330153942108
Loss: 0.20826172828674316
Loss: 0.24966277182102203
Loss: 0.14939840137958527
Loss: 0.08269080519676208
Loss: 0.1417185217142105
Loss: 0.2992289662361145
Loss: 0.076466046273708

Evaluating QA tests: 100%|██████████| 211/211 [00:01<00:00, 177.63it/s]


Model Performance Metrics for epoch:0
----------------------------
Avg loss 0.17829052548988963
Average Cell Precision: 0.4940521327014217
Average Cell Recall: 0.46686255924170617
Average Tuple Cardinality Accuracy: 0.5622654028436018
Average Tuple Constraint Accuracy: 0.3462132701421801
Average Tuple Order Accuracy: 0.1919186046511628
----------------------------
Epoch: 1
Loss: 0.31104400753974915
Loss: 0.1766645610332489
Loss: 0.25600484013557434
Loss: 0.3075356185436249
Loss: 0.11786364018917084
Loss: 0.07012173533439636
Loss: 0.06811198592185974
Loss: 0.1704719066619873
Loss: 0.1965348869562149
Loss: 0.23852695524692535
Loss: 1.1572271585464478
Loss: 0.9371020793914795
Loss: 0.196760356426239
Loss: 0.040910374373197556
Loss: 0.08518311381340027
Loss: 0.2805955410003662
Loss: 0.28919267654418945
Loss: 0.9577988386154175
Loss: 0.3055041432380676
Loss: 0.21446006000041962
Loss: 0.14052298665046692
Loss: 0.13969476521015167
Loss: 0.41955265402793884
Loss: 0.22854328155517578
Loss: 0.25

Evaluating QA tests: 100%|██████████| 211/211 [00:01<00:00, 162.17it/s]


...

Model Performance Metrics for epoch:2
----------------------------
Avg loss 0.12862986614155908
Average Cell Precision: 0.5695260663507108
Average Cell Recall: 0.5389715639810426
Average Tuple Cardinality Accuracy: 0.7173744075829385
Average Tuple Constraint Accuracy: 0.4132464454976303
Average Tuple Order Accuracy: 0.3777872340425532
----------------------------
Epoch: 3
Loss: 0.0790642499923706
Loss: 0.1709599494934082
Loss: 0.0916530191898346
Loss: 0.0968795120716095
Loss: 0.07069537043571472
Loss: 0.0758436769247055
Loss: 0.05669151619076729
Loss: 0.08795515447854996
Loss: 0.13661795854568481
Loss: 0.3160078525543213
Loss: 0.2962087094783783
Loss: 1.2873340845108032
Loss: 1.7157717943191528
Loss: 0.04090167582035065
Loss: 0.026247959583997726
Loss: 0.8403841853141785
Loss: 0.1001235619187355
Loss: 0.592146635055542
Loss: 0.14729760587215424
Loss: 0.05205288529396057
Loss: 0.16492082178592682
Loss: 0.33467087149620056
Loss: 0.22546738386154175
Loss: 0.13006098568439484
Loss: 0.07568

Evaluating QA tests: 100%|██████████| 211/211 [00:01<00:00, 144.34it/s]


Model Performance Metrics for epoch:3
----------------------------
Avg loss 0.09877846795696703
Average Cell Precision: 0.6298862559241707
Average Cell Recall: 0.5867440758293838
Average Tuple Cardinality Accuracy: 0.7818625592417061
Average Tuple Constraint Accuracy: 0.4499620853080569
Average Tuple Order Accuracy: 0.4232051282051283
----------------------------
Epoch: 4
Loss: 0.16605469584465027
Loss: 0.4564093053340912
Loss: 0.14514847099781036
Loss: 0.11454199999570847
Loss: 0.04300885647535324
Loss: 0.04206852987408638
Loss: 0.04071690887212753
Loss: 0.023148544132709503
Loss: 0.08495932817459106
Loss: 0.2567107081413269
Loss: 0.28128084540367126
Loss: 0.056539952754974365
Loss: 0.27164334058761597
Loss: 0.04348409175872803
Loss: 0.06394781172275543
Loss: 0.10109410434961319
Loss: 0.4189787209033966
Loss: 0.10071291029453278
Loss: 0.31732064485549927
Loss: 0.053131211549043655
Loss: 0.053133439272642136
Loss: 0.046051274985075
Loss: 0.24045774340629578
Loss: 0.01575149968266487
Lo

Evaluating QA tests: 100%|██████████| 211/211 [00:01<00:00, 152.17it/s]

Model Performance Metrics for epoch:4
----------------------------
Avg loss 0.07836697370702077
Average Cell Precision: 0.6337345971563982
Average Cell Recall: 0.5805308056872037
Average Tuple Cardinality Accuracy: 0.7539241706161137
Average Tuple Constraint Accuracy: 0.4607298578199052
Average Tuple Order Accuracy: 0.3511702127659575
----------------------------


In [ ]:
stats_df

,epoch,avg_loss,avg_cell_precision,avg_cell_recall,avg_tuple_cardinality,avg_tuple_constraint,avg_tuple_order
0,0,0.178291,0.494052,0.466863,0.562265,0.346213,0.191919
1,1,0.156995,0.629052,0.598607,0.743962,0.467355,0.375091
2,2,0.128630,0.569526,0.538972,0.717374,0.413246,0.377787
3,3,0.098778,0.629886,0.586744,0.781863,0.449962,0.423205
4,4,0.078367,0.633735,0.580531,0.753924,0.460730,0.351170


In [ ]:
#@title Saving the model
save_path = '/content/drive/MyDrive/model_checkpoints'

# Save the model and optimizer state at the end of training
if not os.path.exists(save_path):
    os.makedirs(save_path)
checkpoint_path = os.path.join(save_path, "model_spiderQatch2000.pt")
torch.save({
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
}, checkpoint_path)
print(f"Final checkpoint saved to {checkpoint_path}")

Final checkpoint saved to /content/drive/MyDrive/model_checkpoints/Spider100prctFinal.pt


In [ ]:
#@title Loading an existing model
import torch
import os


save_path = '/content/drive/MyDrive/model_checkpoints'
checkpoint_path = os.path.join(save_path, "100prct_spider_checkpoint4.pt")
# checkpoint_path = os.path.join(save_path,"qatch_spider_checkpoint6000.pt")


if os.path.exists(checkpoint_path):
    # Load the checkpoint
    checkpoint = torch.load(checkpoint_path)#,map_location=torch.device('cpu')

    # Restore the model and optimizer states
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    print("Checkpoint loaded successfully.")
else:
    print(f"No checkpoint found at {checkpoint_path}")


In [ ]:
spider_valid_pd['predictions_TAPAS'] = spider_valid_pd.apply(getAnswerValid, axis=1)

Processing row 0...
Row 0 processed successfully.
Processing row 1...
Row 1 processed successfully.
Processing row 2...
Row 2 processed successfully.
Processing row 3...
Row 3 processed successfully.
Processing row 8...
Row 8 processed successfully.
Processing row 9...
Row 9 processed successfully.
Processing row 14...
Row 14 processed successfully.
Processing row 15...
Row 15 processed successfully.
Processing row 20...
Row 20 processed successfully.
Processing row 21...
Row 21 processed successfully.
Processing row 87...
Row 87 processed successfully.
Processing row 88...
Row 88 processed successfully.
Processing row 91...
Row 91 processed successfully.
Processing row 92...
Row 92 processed successfully.
Processing row 179...
Row 179 processed successfully.
Processing row 180...
Row 180 processed successfully.
Processing row 181...
Row 181 processed successfully.
Processing row 182...
Row 182 processed successfully.
Processing row 183...
Row 183 processed successfully.
Processing row

In [ ]:
tests_df_results = evaluator.evaluate_with_df(spider_valid_pd,
                                      prediction_col_name=f'predictions_TAPAS',
                                      task="QA")

Evaluating QA tests: 100%|██████████| 216/216 [00:12<00:00, 17.49it/s]


In [ ]:
stats_df = tests_df_results[['cell_precision_predictions_TAPAS', 'cell_recall_predictions_TAPAS', 'tuple_cardinality_predictions_TAPAS', 'tuple_constraint_predictions_TAPAS', 'tuple_order_predictions_TAPAS']].describe()

mean_cell_precision = tests_df_results['cell_precision_predictions_TAPAS'].mean()
mean_cell_recall = tests_df_results['cell_recall_predictions_TAPAS'].mean()
mean_tuple_cardinality = tests_df_results['tuple_cardinality_predictions_TAPAS'].mean()
mean_tuple_constraint = tests_df_results['tuple_constraint_predictions_TAPAS'].mean()
mean_tuple_order = tests_df_results['tuple_order_predictions_TAPAS'].mean()

print("Model Performance Metrics:")
print("----------------------------")
print("Average Cell Precision:", mean_cell_precision)
print("Average Cell Recall:", mean_cell_recall)
print("Average Tuple Cardinality Accuracy:", mean_tuple_cardinality)
print("Average Tuple Constraint Accuracy:", mean_tuple_constraint)
print("Average Tuple Order Accuracy:", mean_tuple_order)

Model Performance Metrics:
----------------------------
Average Cell Precision: 0.5218703703703703
Average Cell Recall: 0.48767129629629624
Average Tuple Cardinality Accuracy: 0.5622499999999998
Average Tuple Constraint Accuracy: 0.3778703703703704
Average Tuple Order Accuracy: 0.21689473684210528
